## Answering Business Questions using SQL

In this project, we're going to practice using our SQL skills to answer business questions.


### 1.  Connect to database

In [9]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

### 2. Overview of the Data
Write a query to return information on the tables and views in the database.

In [10]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


### 3.Selecting Albums to Purchase

We've been tasked with selecting the first three albums that will be
added to the store from following artist, from a list of four:

- Regal (Hip-Hop)
- Red Tone (Punk)
- Meteor and the Girls (Pop)
- Slim Jim Bites (Blues)

We'll write a query that returns each genre, with the number of tracks sold in the USA:
- in absolute numbers
- in percentages.

In [11]:
%%sql

WITH 
tracks_sold_in_us AS
(
    SELECT c.country, il.* FROM invoice_line il
    INNER JOIN invoice iv ON iv.invoice_id = il.invoice_id
    INNER JOIN customer c on c.customer_id = iv.customer_ID
    WHERE c.country ='USA'
),
total_sold AS
(
    SELECT sum(quantity) quantity from tracks_sold_in_us
)

SELECT 
  g.name, 
  sum(t.quantity) track_sold,
  cast(sum(t.quantity) AS FLOAT) /
    (select quantity from total_sold) percentage_sold
  FROM tracks_sold_in_us t
  INNER JOIN track tr ON tr.track_id = t.track_id
  INNER JOIN genre g ON g.genre_id = tr.genre_id
  GROUP BY g.name
  ORDER BY track_sold DESC
  LIMIT 10;

Done.


name,track_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


Based on the above query result we should select the following three albums:

- Red Tone (Punk)
- Slim Jim Bites (Blues)
- Meteor and the Girls (Pop)

### 4. Analyzing Employee Sales Performance

Write a query that finds the total dollar amount of sales assigned to each sales support agent within the company. 

In [12]:
%%sql
SELECT
    e.first_name || ' ' || e.last_name "Employee Name",
    e.hire_date "Hire Date",
    sum(total) "Total Sales"
    from employee e
    INNER JOIN customer c on c.support_rep_id = e.employee_id
    INNER JOIN invoice i on c.customer_id = i.customer_id
    WHERE e.title = "Sales Support Agent"
    GROUP BY 1;

Done.


Employee Name,Hire Date,Total Sales
Jane Peacock,2017-04-01 00:00:00,1731.5100000000039
Margaret Park,2017-05-03 00:00:00,1584.0000000000034
Steve Johnson,2017-10-17 00:00:00,1393.920000000002


The difference of sales roughly corresponds with the differences in their hiring dates.


### 5. Analyzing Sales By Country

The next task is to analyze the sales data for customers from each different country.  We will calculate data, for each country, on the:
- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Where a country has only one customer, collect them into an "Other" group.

In [13]:
%%sql
WITH 
other_country AS (
     SELECT
         CASE  
           WHEN (
               SELECT count(*)
               FROM customer
               WHERE country =  c.country
               ) =1 THEN 'Other' 
           ELSE c.country
         END AS country,
         customer_id
     FROM customer c
),
Sales_By_Country AS (
     SELECT
         ot.country,
         count(Distinct ot.customer_id) total_customer,
         sum(i.total) total_sales,
         count(i.invoice_id) total_order,
         CASE  
           WHEN country= 'Other' THEN 1 
           ELSE 0
         END sort
     FROM other_country ot
     INNER JOIN invoice i ON i.customer_id=ot.customer_id
     GROUP by  ot.country
)
SELECT 
    country,
    total_customer,
    Round(total_sales,2) total_sales,
    Round(total_sales / total_customer,2) avg_sales_per_customer,
    Round(total_sales / total_order,2) avg_order_value
    FROM Sales_By_Country
    ORDER by sort ASC, total_sales desc;

Done.


country,total_customer,total_sales,avg_sales_per_customer,avg_order_value
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.57,8.72
Other,15,1094.94,73.0,7.45


Based on the average order value, there may be opportunity in the following countries:

- Czech Republic
- United Kingdom
- India


### 6. Albums vs individual Tracks
The  new strategy that management  are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [14]:
%%sql
WITH
invoice_album AS
( 
SELECT iv.invoice_id,
       t.album_id,
       count (distinct il.track_id) tracks
    FROM invoice iv
    INNER JOIN invoice_line il ON il.invoice_id = iv.invoice_id
    INNER JOIN track t ON t.track_id = il.track_id
    GROUP by iv.invoice_id, album_id
    ORDER BY iv.invoice_id
),
invoice_top_album As
(
SELECT invoice_id ,
    album_id,
    MAX(tracks) tracks
    FROM  invoice_album
    GROUP by invoice_id
    ORDER BY invoice_id
),
tracks_per_album AS 
(
SELECT album_id, COUNT(track_id) AS "track_count"
      FROM track
      GROUP BY 1
),
invoice_album_purchase AS
(
SELECT 
    it.invoice_id,
    CASE  
       WHEN it.tracks = (
         SELECT tp.track_count 
           FROM tracks_per_album tp
           WHERE tp.album_id = it.album_id
         ) THEN 'yes'
        ELSE 'no'
        END album_purchased
    from invoice_top_album it
)  

SELECT
    album_purchased,
    count(invoice_id) number_of_invoice,
    CAST(count(album_purchased) AS FLOAT)/
           (select count(*) from invoice)   percent  
    FROM invoice_album_purchase ia
    GROUP BY album_purchased

Done.


album_purchased,number_of_invoice,percent
no,498,0.8110749185667753
yes,116,0.18892508143322476


Album purchases account for 18.9% of purchases. Based on this data, we would recommend against the new strategy that management proposed. There is potential to lose one fifth of revenue.